In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv("data/tmdb_5000_movies.csv")
credits = pd.read_csv("data/tmdb_5000_credits.csv")

In [3]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [4]:
credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [5]:
movies = movies.merge(right = credits, how = 'inner', on = 'title')

In [6]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

### I need to create tags, so would select those columns which I feel is useful for doing so.
* ID, Genre, keywords, title, overview, release_date, cast, crew

In [7]:
movies = movies[['id', 'title', 'genres', 'keywords', 'overview', 'release_date', 'cast', 'crew']]

In [8]:
movies.isnull().sum()

id              0
title           0
genres          0
keywords        0
overview        3
release_date    1
cast            0
crew            0
dtype: int64

### Since, these are not very big numbers it is ok to drop these rows.

In [9]:
movies.dropna(inplace = True)

In [10]:
movies.duplicated().sum()

np.int64(0)

### No duplicate rows

In [11]:
movies.genres[0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

### As we can see above, it is in a very or not useful format
### We need to get this in `[val1, val2, val3, ...]` format

In [12]:
import json

In [13]:
json_str = '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"},{"id": 14, "name": "Fantasy"}]'

In [14]:
pd.DataFrame(json.loads(json_str))['name'].tolist()

['Action', 'Adventure', 'Fantasy']

In [15]:
import ast
pd.DataFrame(ast.literal_eval(json_str))['name'].tolist()

['Action', 'Adventure', 'Fantasy']

In [16]:
def convert_to_array_ast(genre):

    # eval() or ast's literal_eval
    L = []
    for i in ast.literal_eval(genre):
        L.append(i['name'])
    return L
    # first = eval(genre)
    # second = pd.DataFrame(first)
    # third = second['name'].tolist()

    # return third

In [17]:
convert_to_array_ast(json_str)

['Action', 'Adventure', 'Fantasy']

In [18]:
def convert_to_array_json(genre):

    L = []
    for i in json.loads(genre):
        L.append(i['name'])
    return L
    # first = json.loads(genre)
    # second = pd.DataFrame(first)
    # third = second['name'].tolist()

    # return third

In [19]:
convert_to_array_json(json_str)

['Action', 'Adventure', 'Fantasy']

In [20]:
# movies['genres'].apply(convert_to_array_ast)

In [21]:
movies['genres'] = movies['genres'].apply(convert_to_array_ast)

In [22]:
movies.head(2)

,id,title,genres,keywords,overview,release_date,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",2009-12-10,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",2007-05-19,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [23]:
movies['keywords']

0       [{"id": 1463, "name": "culture clash"}, {"id":...
1       [{"id": 270, "name": "ocean"}, {"id": 726, "na...
2       [{"id": 470, "name": "spy"}, {"id": 818, "name...
3       [{"id": 849, "name": "dc comics"}, {"id": 853,...
4       [{"id": 818, "name": "based on novel"}, {"id":...
                              ...                        
4804    [{"id": 5616, "name": "united states\u2013mexi...
4805                                                   []
4806    [{"id": 248, "name": "date"}, {"id": 699, "nam...
4807                                                   []
4808    [{"id": 1523, "name": "obsession"}, {"id": 224...
Name: keywords, Length: 4805, dtype: object

In [24]:
movies['keywords'] = movies['keywords'].apply(convert_to_array_json)

In [25]:
movies['keywords'][1]

['ocean',
 'drug abuse',
 'exotic island',
 'east india trading company',
 "love of one's life",
 'traitor',
 'shipwreck',
 'strong woman',
 'ship',
 'alliance',
 'calypso',
 'afterlife',
 'fighter',
 'pirate',
 'swashbuckler',
 'aftercreditsstinger']

In [26]:
movies['cast'].apply(convert_to_array_json).apply(lambda x : x[:3])

0        [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1           [Johnny Depp, Orlando Bloom, Keira Knightley]
2            [Daniel Craig, Christoph Waltz, Léa Seydoux]
3            [Christian Bale, Michael Caine, Gary Oldman]
4          [Taylor Kitsch, Lynn Collins, Samantha Morton]
                              ...                        
4804    [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...
4805         [Edward Burns, Kerry Bishé, Marsha Dietlein]
4806           [Eric Mabius, Kristin Booth, Crystal Lowe]
4807            [Daniel Henney, Eliza Coupe, Bill Paxton]
4808    [Drew Barrymore, Brian Herzlinger, Corey Feldman]
Name: cast, Length: 4805, dtype: object

In [27]:
# a = [[3, 1, 4], [2, 5, 0], [7, 6, 8]]

# L = []
# for i in range(len(a)):
#     for j in range(2):
#         L.append(a[i][j])

# print(L)

In [28]:
def main_character(obj):

    counter = 0
    L = []
    for i in json.loads(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

In [29]:
movies['cast'] = movies['cast'].apply(main_character)

In [30]:
movies.head(1)

,id,title,genres,keywords,overview,release_date,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",2009-12-10,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [31]:
# def find_director(obj):
#     L = []
#     for i in range(len(obj)):
#         L.append([c['name'] for c in obj.apply(lambda x : json.loads(x))[i] if c['job'] == 'Director'])
#     return L

In [32]:
[c['name'] for c in movies['crew'].apply(lambda x : json.loads(x))[1] if c['job'] == 'Director']

['Gore Verbinski']

In [33]:
def find_director(obj):

    L = []
    for i in json.loads(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [34]:
movies['crew'] = movies['crew'].apply(find_director)

In [35]:
movies.head(2)

,id,title,genres,keywords,overview,release_date,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",2009-12-10,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",2007-05-19,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [36]:
# a = [{'a' : 1, 'b' : 2}, {'c' : 3, 'b' : 4}, {'e' : 5, 'b' : 6}]

# bs = [c['b'] for c in a]
# bs

In [37]:
# pd.DataFrame(movies['crew'].apply(lambda x : json.loads(x))[6])['job'].value_counts()['Director']

In [38]:
# for i in range(10):
#     print(pd.DataFrame(movies['crew'].apply(lambda x : json.loads(x))[i])['job'].value_counts()['Director'])

# len(movies['crew'])

In [39]:
movies['overview'] = movies['overview'].apply(lambda x : x.split())

In [40]:
movies.head(2)

,id,title,genres,keywords,overview,release_date,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[In, the, 22nd, century,, a, paraplegic, Marin...",2009-12-10,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Captain, Barbossa,, long, believed, to, be, d...",2007-05-19,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


# Removes spaces

In [41]:
'Sam Worthington'.replace(" ", "")

'SamWorthington'

In [42]:
movies['cast'] = movies['cast'].apply(lambda x: [item.replace(" ", "") for item in x])

In [43]:
movies['genres'] = movies['genres'].apply(lambda x: [item.replace(" ", "") for item in x])

In [44]:
movies['keywords'] = movies['keywords'].apply(lambda x: [item.replace(" ", "") for item in x])

In [45]:
movies['crew'] = movies['crew'].apply(lambda x: [item.replace(" ", "") for item in x])

In [46]:
movies.head(2)

,id,title,genres,keywords,overview,release_date,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",2009-12-10,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",2007-05-19,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]


# Creating tags by concatenating columns

In [47]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [48]:
df = movies[['id', 'title', 'tags']]

In [49]:
df.head()

,id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [50]:
df['tags'] = df['tags'].apply(lambda x : " ".join(x))

/tmp/ipykernel_6932/1642297555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x : " ".join(x))


In [51]:
df.head()

,id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [52]:
df['tags'] = df['tags'].apply(lambda x : x.lower())

/tmp/ipykernel_6932/4152385510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x : x.lower())


# Vectorization

In [53]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [54]:
vectors = cv.fit_transform(df['tags']).toarray()

In [55]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [56]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

# Stemming

In [57]:
# !pip install nltk

In [58]:
import nltk

In [59]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [60]:
def stem(l):
    y = []
    for i in l.split():
        y.append(ps.stem(i))

    return ' '.join(y)

In [61]:
df['tags'] = df['tags'].apply(stem)

/tmp/ipykernel_6932/866399325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(stem)


In [62]:
df['tags'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [63]:
df['tags'][100]

'tell the stori of benjamin button, a man who start age backward with bizarr consequences. fantasi drama thriller mysteri romanc diari navi funer tea travel hospit cateblanchett bradpitt tildaswinton davidfinch'

In [64]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

### We need to use cosine similarity when we are dealing with high-dimensional data
### cosine similarity calculates the angle between two vectors
### euclidean distance fails on high-dimensional data, and it calculates tip-to-tip distance of two vectors

# Similarity

In [65]:
from sklearn.metrics.pairwise import cosine_similarity

In [66]:
similarity = cosine_similarity(vectors)

In [67]:
def recommend(movie):
    index = df[df['title'] == movie].index[0]

    # enumerate is used to not lose the index values
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(df.iloc[i[0]].title)

In [80]:
recommend('Spider-Man')

Spider-Man 3
Spider-Man 2
The Amazing Spider-Man 2
The Amazing Spider-Man
Arachnophobia


In [69]:
import pickle

In [70]:
matching_movies = sorted(list(enumerate(similarity[df[df['title'] == 'Batman Begins'].index[0]])), reverse = True, key = lambda x : x[1])

In [71]:
for i in matching_movies[1:6]:
    print(df.iloc[i[0]].title)

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Batman


In [110]:
pickle.dump(df, open('movie_list.pkl','wb'))
pickle.dump(similarity, open('similarity.pkl','wb'))

In [72]:
pickle.dump(df.to_dict(), open('movie_dict.pkl', 'wb'))